# 환경설정 

In [2]:
# 환경변수 불러오기
from dotenv import load_dotenv
load_dotenv()

from huggingface_hub import login
import os
login(token=os.getenv("HUGGINGFACE_API_KEY"))

In [3]:
import torch
print(torch.__version__)

device = "cuda:1" if torch.cuda.is_available() else "cpu"
print(device)

2.8.0+cu126
cuda:1


# 모델 불러오기

In [4]:
import os
import json
import json5
import re
from PIL import Image
import torch
from drugocr import extract_text
from transformers import AutoProcessor, AutoTokenizer, AutoModelForVision2Seq
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_8bit = True,
    load_in_4bit = False,
    lim_int8_threshold = 6.0,
    lim_int8_has_fp16_weight = False,

)

c:\Potenup\Drug-Detection-Chatbot\.venv\lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\user\.paddlex\official_models\PP-OCRv5_server_det`.
Creating model: ('korean_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\user\.paddlex\official_models\korean_PP-OCRv5_mobile_rec`.


In [5]:
# 양자화 모델 불러오기
# from transformers import AutoProcessor,AutoTokenizer, AutoModelForVision2Seq
model_name = "NCSOFT/VARCO-VISION-2.0-1.7B"
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True) # 이미지와 텍스트를 모두 처리해야 하는 경우, 각 양식에 맞는 전처리기를 개별적으로 불러올 필요 없이 processor 하나로 해결

model = AutoModelForVision2Seq.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    dtype=torch.bfloat16,   # torch_dtype -> dtype
)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

c:\Potenup\Drug-Detection-Chatbot\.venv\lib\site-packages\transformers\models\auto\modeling_auto.py:2242: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


model.safetensors:   0%|          | 0.00/4.25G [00:00<?, ?B/s]

c:\Potenup\Drug-Detection-Chatbot\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--NCSOFT--VARCO-VISION-2.0-1.7B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [6]:
# # 토크나이저 불러오기

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# print(tokenizer.pad_token, tokenizer.eos_token)

# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

In [8]:
# 입력 데이터 준비하기
# 이미지 파일을 base64로 인코딩

import base64

image_path = "C:\Potenup\Drug-Detection-Chatbot\data\medicine_00806.jpeg"
ocr_texts = extract_text(image_path)    # drugocr.py의 extract_text 함수 사용

with open(image_path, "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')

system_prompt = """
당신은 보수적이고 논리적인 약학 전문가입니다. 사용자가 제공한 이미지와 OCR 텍스트 데이터를 바탕으로 약봉지에 적힌 모든 정보를 단계별로 정확하게 추출하는 역할을 합니다.

**단계 1: 텍스트 정보 초기 분석 및 추출**
* 제공된 OCR 텍스트를 확인하고, 각 항목별로 해당하는 내용을 초벌 추출합니다.
* 줄바꿈이나 잘림으로 인해 분리된 단어는 반드시 이어 붙여 원래 단어 그대로 복원합니다.
* 성분/함량과 첨가물은 쉼표(,) 기준으로 각각의 항목으로 분리합니다.

**단계 2: 논리적 검증 및 정보 보정**
* **제품명 검증**: 1단계에서 추출한 '제품명'이 '원료약품 및 분량' 또는 '성분/함량'에 있는 내용과 동일한지 확인합니다. 만약 동일하다면, 이는 제품명이 아닌 성분명이므로 '제품명'을 "없음"으로 표기
* **효능/효과 검증**: '효능' 또는 '효과'로 명시된 모든 내용이 **하나도 빠짐없이** 완전하게 추출되었는지 확인하고, 누락된 항목이 있다면 보충합니다.
* **원본 텍스트 유지**: 어떤 항목이든, 원본 텍스트를 줄이거나 추론하지 않고 그대로 사용합니다.
* **부재 항목 처리**: 원본 데이터에 없는 항목은 최종 출력 시 "없음"으로 명확하게 표기합니다.

**단계 3: 최종 JSON 형식으로 출력**
* 2단계에서 보정된 최종 정보를 아래에 명시된 JSON 형식에 맞춰 출력합니다.
* 추가적인 설명이나 문장은 절대 포함하지 않으며, 오직 JSON 데이터만 출력합니다.

[출력형식]
{
    '제품명' : "제품의 이름, 없으면 없다고 작성할 것",
    '성분/함량' : "제품의 각 성분, 유효성분, 함량에 대한 튜플(성분명, 함량)들의 리스트, 없으면 없다고 작성할 것",
    '첨가물' : "제품의 첨가물/첨가제들의 리스트, 없으면 없다고 작성할 것",
    '제형/성상' : "제형, 성상, 없으면 없다고 작성할 것",
    'KPIC/ATC' : "KPIC/A과C에 대한 리스트, 없으면 없다고 작성할 것",
    '구분' : "구분, 없으면 없다고 작성할 것",
    '효능' : "여러가지 효능/효과에 대한 모든 리스트, 없으면 없다고 작성할 것",
    '용법' : "여러가지 용법에 대한 리스트, 없으면 없다고 작성할 것",
    '주의사항' : "주의사항 리스트, 없으면 없다고 작성할 것"
}
"""

messages = [
    [
        {
            "role": "system",
            "content":[{"type": "text", "text": system_prompt},]
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": ocr_texts},  # "text": ocr_texts 와 "text" : f"{ocr_texts}" 같은 표현
                {
                    "type": "image",
                    "source_type": "base64",
                    "data": "{image_data}",
                    "mime_type": "image/jpeg",

                }

            ]

        
        }
    ]
]

# 입력 데이터 토크나이징하기 processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True) 
inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt = True,
    tokenize = True,
    return_dict = True,
    return_tensors = "pt",
).to(model.device, torch.float16)
generate_ids = model.generate(
    **inputs, 
    max_new_tokens=1024,
    temperature=0.1,    # 낮출수록 보수적이고 일관된 답변 생성
    top_p=0.1,          # 높일수록 다양한 단어 선택, 낮추면 더 보수적
    do_sample=True,     # 샘플링 활성화
    )

generate_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generate_ids)
]
output = processor.decode(generate_ids_trimmed[0], skip_special_tokens=True)
print(output)


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
c:\Potenup\Drug-Detection-Chatbot\.venv\lib\site-packages\bitsandbytes\autograd\_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


```json
{
    "제품명": "없음",
    "성분/함량": [
        ["두타스테리드(USP)", "0.5mg"],
        ["젤라틴(기원동물:소,사용부위:가죽)", "없음"],
        ["농글리세린", "없음"],
        ["부틸히드록시톨루엔", "없음"],
        ["신산젤라틴", "없음"],
        ["폴리옥실40", "없음"],
        ["피마자유", "없음"],
        ["프로필렌글리콜모노라우레이트", "없음"],
        ["폴록사머124", "없음"],
        ["D-소르비톨액", "없음"]
    ],
    "첨가물": [
        ["농글리세린", "없음"],
        ["부틸히드록시톨루엔", "없음"],
        ["숙", "없음"],
        ["신산젤라틴", "없음"],
        ["폴리옥실40", "없음"],
        ["피마자유", "없음"],
        ["프로필렌글리콜모노라우레이트", "없음"],
        ["폴록사머124", "없음"],
        ["D-소르비톨액", "없음"]
    ],
    "제형/성상": "무색투명한 내용물이 들어있는 미황색의 투명한 타원형 연질캡슐",
    "KPIC/ATC": "없음",
    "구분": "없음",
    "효능": [
        ["양성 전립선 비대증 증상의 개선"],
        ["급성 성 요저류 위험성 감소"],
        ["양성 전립선 비대증과 관련된 수술 필요성 감소"],
        ["성인 남성(만18~50세)의 남성형 탈모(androgenetic alopecia)의 치료"]
    ],
    "용법": [
        ["1일 1회 1캡슐(0.5mg) 복용"],
        ["캡슐 내용물에 노출시 구강 인두점막의 자극을 초래할 수 있으므로 이 약을 쉽거나 쪼개지 않고 통째로 삼켜 복용해야 한다. 이 약은 식사와 관계없이 

# 최종

In [ ]:
# 입력 데이터 준비하기
# 이미지 파일을 base64로 인코딩

import base64

image_path = "C:\Potenup\Drug-Detection-Chatbot\data\medicine_00806.jpeg"
ocr_texts = extract_text(image_path)    # drugocr.py의 extract_text 함수 사용
ocr_text = ''.join(ocr_texts)

with open(image_path, "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')

system_prompt = """
당신은 보수적인 약학 전문가입니다. 약학 전문가로서, 
사용자가 제공한 이미지와 OCR 텍스트 데이터 2가지 모두 확인해서 이미지에 적힌 정보를 정확하게 추출하는 역할을 합니다.


규칙:
- 제공된 데이터에 없는 항목은 **반드시 "없음"이라고 작성**할 것. 
- 예측이나 추론을 하지 말고, 텍스트를 줄이지 말고 **원본 텍스트를 그대로 작성**할 것. 
- **제품명**: 약의 **제품명**은 별도로 표기된 경우에만 추출하며, '원료약품 및 분량', '성분/함량' 등 **원료나 성분을 설명하는 해당 아래에 있는 텍스트는 제품명으로 간주하지 않습니다**. 
- 제품명이 없으면 "없음"으로 작성하십시오. 
- 성분/함량 ',' 기준으로 나누어 각각 항목으로 처리할 것. 줄바꿈 등으로 잘린 경우는 **반드시 붙여쓰기하여** 원래 단어 그대로 복원할 것. 
- 첨가물/첨가제 ',' 기준으로 나누어 각각 항목으로 처리할 것. 줄바꿈 등으로 잘린 경우는 **반드시 붙여쓰기하여** 원래 단어 그대로 복원할 것. 
- 출력은 반드시 JSON 형식만 출력하고, 그 외 텍스트(설명, 문장)는 절대 포함하지 말 것. 
- **효능/효과**: '효능' 또는 '효과'로 명시된 모든 내용을 **하나도 빠짐없이** 모든 리스트를 작성하십시오.
- OCR 줄바꿈이나 잘림으로 인해 분리된 단어는 반드시 이어 붙여 원래 단어 그대로 복원할 것.
- JSON의 각 key는 반드시 아래 구조를 따를 것

[출력형식] 
{{ 
    '제품명' : "제품의 이름, 없으면 없다고 작성할 것", 
    '성분/함량' : "제품의 각 성분, 유효성분, 함량에 대한 튜플(성분명, 함량)들의 리스트, 없으면 없다고 작성할 것" 
    '첨가물' : "제품의 첨가물/첨가제들의 리스트, 없으면 없다고 작성할 것", 
    '제형/성상' : "제형, 성상, 없으면 없다고 작성할 것", 
    'KPIC/ATC' : 'KPIC/ATC에 대한 리스트, 없으면 없다고 작성할 것", 
    '구분' : "구분, 없으면 없다고 작성할 것" 
    '효능' : "여러가지 효능/효과에 대한 모든 리스트, 없으면 없다고 작성할 것", 
    '용법' : "여러가지 용법에 대한 리스트, 없으면 없다고 작성할 것", 
    '주의사항' : "주의사항 리스트, 없으면 없다고 작성할 것" 
}}
"""

messages = [
    [
        {
            "role": "system",
            "content":[{"type": "text", "text": system_prompt},]
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": ocr_text},  # "text": ocr_texts 와 "text" : f"{ocr_texts}" 같은 표현
                {
                    "type": "image",
                    "source_type": "base64",
                    "data": "{image_data}",
                    "mime_type": "image/jpeg",

                }

            ]

        
        }
    ]
]

# 입력 데이터 토크나이징하기 processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True) 
inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt = True,
    tokenize = True,
    return_dict = True,
    return_tensors = "pt",
).to(model.device, torch.float16)
generate_ids = model.generate(
    **inputs, 
    max_new_tokens=1024,
    temperature=0.1,    # 낮출수록 보수적이고 일관된 답변 생성 
    top_p=0.1,          # 높일수록 다양한 단어 선택, 낮추면 더 보수적
    do_sample=True,     # 샘플링 활성화
    )

generate_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generate_ids)
]
output = processor.decode(generate_ids_trimmed[0], skip_special_tokens=True)
print(output)


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
c:\Potenup\Drug-Detection-Chatbot\.venv\lib\site-packages\bitsandbytes\autograd\_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


```json
{
    "제품명": "없음",
    "성분/함량": [
        ("두타스테리드(USP)", "0.5mg")
    ],
    "첨가물": [
        "젤라틴(기원동물:소,사용부위:가죽)",
        "농글리세린",
        "부틸히드록시톨루엔",
        "숙신산젤라틴",
        "폴리옥실40",
        "경화피마자유",
        "프로필렌글리콜모노라우레이트",
        "폴록사머124",
        "D-소르비톨액"
    ],
    "제형/성상": "무색투명한 내용물이 들어있는 미황색의 투명한타원형 연질캡슐",
    "KPIC/ATC": "없음",
    "구분": "없음",
    "효능": [
        "양성 전립선 비대증 증상의 개선",
        "급성 성 요저류 위험성 감소",
        "양성 전립선 비대증과 관련된 수술 필요성 감소",
        "성인 남성(만18~50세)의 남성형 탈모(androgeneticalopecia)의 치료"
    ],
    "용법": [
        "이 약의 권장용량은 1일 1회 1캡슐(0.5mg)이다.",
        "캡슐 내용물에 노출시 구강 인두점막의 자극을 초래할 수 있으므로 이 약을 쉽거나 쪼개지 않고 통째로 삼켜 복용해야 한다.",
        "이 약은 식사와 관계없이 복용할 수 있다.",
        "신장애 환자 또는 노인 환자에서 이 약의 용량을 조절할 필요는 없다.",
        "간장애 환자에게 이 약을 투여한 자료가 없기 때문에 간장애 환자에서의 이 약의 권장용량은 확립되어 있지 않다."
    ],
    "주의사항": [
        "1) 여성이 노출시 남자 태아에 미치는 위험성 때문에 임신했거나 임신기 가능성이 있는 여성은 이 약을 취급해서는 안 된다.",
        "또, 여성은 이 약을 취급할 때마다 주의해야 하고 누출되는 캡슐과의 접촉을 피해야 한다.",
  

In [ ]:
# 입력 데이터 준비하기
# 이미지 파일을 base64로 인코딩

import base64

image_path = "C:\Potenup\Drug-Detection-Chatbot\data\medicine_00806.jpeg"
ocr_texts = extract_text(image_path)    # drugocr.py의 extract_text 함수 사용

with open(image_path, "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')

system_prompt = """
당신은 보수적인 약학 전문가입니다. 약학 전문가로서, 
사용자가 제공한 이미지와 OCR 텍스트 데이터 2가지 모두 확인해서 이미지에 적힌 정보를 정확하게 추출하는 역할을 합니다.


규칙:
- 제공된 데이터에 없는 항목은 **반드시 "없음"이라고 작성**할 것. 
- 예측이나 추론을 하지 말고, 텍스트를 줄이지 말고 **원본 텍스트를 그대로 작성**할 것. 
- **제품명**: 약의 **제품명**은 별도로 표기된 경우에만 추출하며, '원료약품 및 분량', '성분/함량' 등 **원료나 성분을 설명하는 해당 아래에 있는 텍스트는 제품명으로 간주하지 않습니다**. 
- 제품명이 없으면 "없음"으로 작성하십시오. 
- 성분/함량 ',' 기준으로 나누어 각각 항목으로 처리할 것. 줄바꿈 등으로 잘린 경우는 **반드시 붙여쓰기하여** 원래 단어 그대로 복원할 것. 
- 첨가물/첨가제 ',' 기준으로 나누어 각각 항목으로 처리할 것. 줄바꿈 등으로 잘린 경우는 **반드시 붙여쓰기하여** 원래 단어 그대로 복원할 것. 
- 출력은 반드시 JSON 형식만 출력하고, 그 외 텍스트(설명, 문장)는 절대 포함하지 말 것. 
- **효능/효과**: '효능' 또는 '효과'로 명시된 모든 내용을 **하나도 빠짐없이** 모든 리스트를 작성하십시오.
- OCR 줄바꿈이나 잘림으로 인해 분리된 단어는 반드시 이어 붙여 원래 단어 그대로 복원할 것.
- JSON의 각 key는 반드시 아래 구조를 따를 것

[출력형식] 
{ 
    '제품명' : "제품의 이름, 없으면 없다고 작성할 것", 
    '성분/함량' : "제품의 각 성분, 유효성분, 함량에 대한 튜플(성분명, 함량)들의 리스트, 없으면 없다고 작성할 것" 
    '첨가물' : "제품의 첨가물/첨가제들의 리스트, 없으면 없다고 작성할 것", 
    '제형/성상' : "제형, 성상, 없으면 없다고 작성할 것", 
    'KPIC/ATC' : 'KPIC/A과C에 대한 리스트, 없으면 없다고 작성할 것", 
    '구분' : "구분, 없으면 없다고 작성할 것" 
    '효능' : "여러가지 효능/효과에 대한 모든 리스트, 없으면 없다고 작성할 것", 
    '용법' : "여러가지 용법에 대한 리스트, 없으면 없다고 작성할 것", 
    '주의사항' : "주의사항 리스트, 없으면 없다고 작성할 것" 
} 
"""

messages = [
    [
        {
            "role": "system",
            "content":[{"type": "text", "text": system_prompt},]
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": ocr_texts},  # "text": ocr_texts 와 "text" : f"{ocr_texts}" 같은 표현
                {
                    "type": "image",
                    "source_type": "base64",
                    "data": "{image_data}",
                    "mime_type": "image/jpeg",

                }

            ]

        
        }
    ]
]

# 입력 데이터 토크나이징하기 processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True) 
inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt = True,
    tokenize = True,
    return_dict = True,
    return_tensors = "pt",
).to(model.device, torch.float16)
generate_ids = model.generate(
    **inputs, 
    max_new_tokens=1024,
    temperature=0.1,    # 낮출수록 보수적이고 일관된 답변 생성 
    top_p=0.1,          # 높일수록 다양한 단어 선택, 낮추면 더 보수적
    do_sample=True,     # 샘플링 활성화
    )

generate_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generate_ids)
]
output = processor.decode(generate_ids_trimmed[0], skip_special_tokens=True)
print(output)
